<a href="https://colab.research.google.com/github/gift-framework/GIFT/blob/main/G2_ML/1_3/K7_GIFT_v1_3_zero_param.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K7 GIFT v1.3 Zero-Parameter Framework

Torsion-free G2 metric on TCS K7 with zero phenomenological parameters.

## 1. Imports

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import json
from pathlib import Path
from typing import Dict, Tuple
from dataclasses import dataclass

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float64)
print(f'K7 GIFT v1.3 | Device: {device}')

K7 GIFT v1.3 | Device: cuda


## 2. Structural Constants

In [2]:
@dataclass
class StructuralConstants:
    """Immutable structural constants from E8/G2/K7 geometry."""
    p2: int = 2
    N_gen: int = 3
    Weyl_factor: int = 5
    dim_K7: int = 7
    rank_E8: int = 8
    dim_G2: int = 14
    dim_E8: int = 248
    b2_target: int = 21
    b3_target: int = 77

    @property
    def H_star(self) -> int:
        return 1 + self.b2_target + self.b3_target

    @property
    def M5(self) -> int:
        return 31

SC = StructuralConstants()
print(f'p2={SC.p2}, N_gen={SC.N_gen}, b2={SC.b2_target}, b3={SC.b3_target}, H*={SC.H_star}')

p2=2, N_gen=3, b2=21, b3=77, H*=99


## 3. Zero-Parameter Physics

In [3]:
class ZeroParamPhysics:
    """All observables from structural constants only."""

    def __init__(self, sc: StructuralConstants):
        self.sc = sc

    @property
    def kappa_T(self) -> float:
        """KAPPA_T: 1/(b3 - dim(G2) - p2) = 1/61"""
        return 1.0 / (self.sc.b3_target - self.sc.dim_G2 - self.sc.p2)

    @property
    def sin2_theta_W(self) -> float:
        """Weinberg angle: b2/(b3 + dim(G2)) = 21/91"""
        return self.sc.b2_target / (self.sc.b3_target + self.sc.dim_G2)

    @property
    def alpha_s_MZ(self) -> float:
        """Strong coupling: sqrt(2)/(dim(G2) - p2) = sqrt(2)/12"""
        return np.sqrt(2) / (self.sc.dim_G2 - self.sc.p2)

    @property
    def lambda_H(self) -> float:
        """Higgs coupling: sqrt(dim(G2) + N_gen)/32 = sqrt(17)/32"""
        return np.sqrt(self.sc.dim_G2 + self.sc.N_gen) / 32

    @property
    def tau_num(self) -> int:
        return (self.sc.p2**4) * self.sc.dim_K7 * self.sc.M5

    @property
    def tau_den(self) -> int:
        return (self.sc.N_gen**4) * (self.sc.rank_E8 + self.sc.N_gen)

    @property
    def tau(self) -> float:
        """TAU-SCALE: 3472/891"""
        return self.tau_num / self.tau_den

    @property
    def beta_0(self) -> float:
        return np.pi / self.sc.rank_E8

    @property
    def xi(self) -> float:
        return (self.sc.Weyl_factor / self.sc.p2) * self.beta_0

ZPP = ZeroParamPhysics(SC)
print(f'kappa_T = 1/61 = {ZPP.kappa_T:.6f}')
print(f'sin2_theta_W = 21/91 = {ZPP.sin2_theta_W:.6f}')
print(f'alpha_s = sqrt(2)/12 = {ZPP.alpha_s_MZ:.6f}')
print(f'tau = {ZPP.tau_num}/{ZPP.tau_den} = {ZPP.tau:.6f}')

kappa_T = 1/61 = 0.016393
sin2_theta_W = 21/91 = 0.230769
alpha_s = sqrt(2)/12 = 0.117851
tau = 3472/891 = 3.896745


## 4. Training Config (Computational Only)

In [4]:
TRAIN_CONFIG = {
    'n_grid': 16,
    'batch_size': 1024,
    'n_fourier': 10,
    'hidden_dim': 256,
    'n_layers': 6,
    'lr_initial': 1e-4,
    'lr_min': 1e-5,
    'warmup_epochs': 200,
    'epochs_per_phase': 2000,
    'print_every': 100,
    'checkpoint_every': 500,
    'tcs': {
        'r_neck_start': 0.35,
        'r_neck_end': 0.65,
        'neck_width': 5.0,
        'twist_angle': np.pi / 3,
    },
    'output_dir': 'outputs_v1_3',
}
Path(TRAIN_CONFIG['output_dir']).mkdir(exist_ok=True)
print('Config loaded')

Config loaded


## 5. Fourier Encoding

In [5]:
class FourierEncoding(nn.Module):
    def __init__(self, n_fourier: int = 10):
        super().__init__()
        self.n_fourier = n_fourier
        self.output_dim = 7 * 2 * n_fourier

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        features = []
        for L in range(1, self.n_fourier + 1):
            features.append(torch.sin(2 * np.pi * L * x))
            features.append(torch.cos(2 * np.pi * L * x))
        return torch.cat(features, dim=-1)

def sample_coords(batch_size: int) -> torch.Tensor:
    return torch.rand(batch_size, 7, device=device, dtype=torch.float64)

print('Fourier encoding ready')

Fourier encoding ready


## 6. PhiNet

In [6]:
class PhiNet(nn.Module):
    def __init__(self, config: Dict):
        super().__init__()
        self.fourier = FourierEncoding(config['n_fourier'])
        hidden = config['hidden_dim']
        layers = [nn.Linear(self.fourier.output_dim, hidden), nn.Tanh()]
        for _ in range(config['n_layers'] - 1):
            layers.extend([nn.Linear(hidden, hidden), nn.Tanh()])
        layers.append(nn.Linear(hidden, 35))
        self.net = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(self.fourier(x))

def to_3form(phi_comp: torch.Tensor) -> torch.Tensor:
    batch = phi_comp.shape[0]
    phi = torch.zeros(batch, 7, 7, 7, device=phi_comp.device, dtype=phi_comp.dtype)
    idx = 0
    for i in range(7):
        for j in range(i+1, 7):
            for k in range(j+1, 7):
                v = phi_comp[:, idx]
                phi[:, i, j, k] = v
                phi[:, i, k, j] = -v
                phi[:, j, i, k] = -v
                phi[:, j, k, i] = v
                phi[:, k, i, j] = v
                phi[:, k, j, i] = -v
                idx += 1
    return phi

print('PhiNet ready')

PhiNet ready


## 7. Metric and Geometry

In [7]:
def phi_to_metric(phi: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
    batch = phi.shape[0]
    g = torch.eye(7, device=phi.device, dtype=phi.dtype).unsqueeze(0).expand(batch, -1, -1).clone()
    for i in range(7):
        for j in range(7):
            g[:, i, j] = g[:, i, j] + 0.1 * (phi[:, i] * phi[:, j]).sum(dim=(1, 2))
    g = 0.5 * (g + g.transpose(-2, -1))
    eigvals = torch.linalg.eigvalsh(g)
    ev, evec = torch.linalg.eigh(g)
    ev = torch.clamp(ev, min=1e-6)
    g = evec @ torch.diag_embed(ev) @ evec.transpose(-2, -1)
    return g, eigvals

class GeometryG2:
    def __init__(self, config: Dict, sc: StructuralConstants):
        self.config = config
        self.sc = sc
        tcs = config['tcs']
        self.r_start = tcs['r_neck_start']
        self.r_end = tcs['r_neck_end']
        self.twist = tcs['twist_angle']
        self.vol_scale = 1.0

    def twist_map(self, x: torch.Tensor) -> torch.Tensor:
        r = x[:, 0]
        rn = torch.clamp((r - self.r_start) / (self.r_end - self.r_start), 0, 1)
        chi = 3 * rn**2 - 2 * rn**3
        xt = x.clone()
        xt[:, 1] = ((2*np.pi*x[:, 1] + chi*self.twist) / (2*np.pi)) % 1.0
        xt[:, 2] = ((2*np.pi*x[:, 2] - chi*self.twist) / (2*np.pi)) % 1.0
        return xt

    def compute_metric(self, net: nn.Module, coords: torch.Tensor):
        phi = to_3form(net(self.twist_map(coords)))
        g, eig = phi_to_metric(phi)
        g = self.vol_scale * g
        return g, {'phi': phi, 'eig': eig, 'det': torch.linalg.det(g)}

print('Geometry ready')

Geometry ready


## 8. Torsion

In [8]:
def ext_deriv(phi: torch.Tensor, coords: torch.Tensor) -> torch.Tensor:
    batch = phi.shape[0]
    dphi = torch.zeros(batch, 7, 7, 7, 7, device=phi.device, dtype=phi.dtype)
    if batch > 1:
        phi_dev = phi - phi.mean(dim=0, keepdim=True)
        for mu in range(7):
            for i in range(7):
                for j in range(i+1, 7):
                    for k in range(j+1, 7):
                        v = phi_dev[:, i, j, k] * (coords[:, mu] - 0.5) * 0.5
                        dphi[:, mu, i, j, k] = v
                        dphi[:, i, mu, j, k] = -v
                        dphi[:, i, j, mu, k] = v
                        dphi[:, i, j, k, mu] = -v
    return dphi

def torsion_norm(dphi: torch.Tensor) -> torch.Tensor:
    return torch.sqrt((dphi ** 2).sum(dim=(1,2,3,4)) + 1e-10)

print('Torsion ready')

Torsion ready


## 9. RG Flow

In [9]:
class RGFlowModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.A = nn.Parameter(torch.tensor(-30.0, dtype=torch.float64))
        self.B = nn.Parameter(torch.tensor(1.0, dtype=torch.float64))
        self.C = nn.Parameter(torch.tensor(20.0, dtype=torch.float64))
        self.D = nn.Parameter(torch.tensor(1.2, dtype=torch.float64))
        self.lam_max = 39.44

    def forward(self, div_T, T_sq, tr_deps, fract):
        total = self.A * div_T + self.B * T_sq + self.C * tr_deps + self.D * fract
        delta = total * self.lam_max / 100
        return delta, {'A': self.A.item(), 'B': self.B.item(), 'C': self.C.item(), 'D': self.D.item()}

print('RG Flow ready')

RG Flow ready


## 10. Loss Functions

In [10]:
def compute_losses(net, geom, coords, rg, zpp, phase):
    g, info = geom.compute_metric(net, coords)
    phi = info['phi']
    det_g = info['det']
    eig = info['eig']

    dphi = ext_deriv(phi, coords)
    T = torsion_norm(dphi)
    T_mean = T.mean()

    losses = {}
    # KAPPA_T target
    losses['torsion'] = (T_mean - zpp.kappa_T) ** 2
    if T_mean > 0.04:
        losses['torsion'] = losses['torsion'] + 5.0 * (T_mean - 0.04) ** 2

    # det(g) = p2 = 2
    losses['det'] = (det_g.mean() - float(geom.sc.p2)) ** 2
    losses['pos'] = torch.relu(-eig.min(dim=-1)[0]).mean()
    losses['harm'] = (phi ** 2).mean() * 0.01

    if phase >= 3:
        da, _ = rg(0.005, T_mean.item()**2, 0.001, -0.5)
        losses['rg'] = (da - (-0.9)) ** 2
    else:
        losses['rg'] = torch.tensor(0.0, device=coords.device)

    w = {1: [0.5, 0.5, 1.0, 0.0, 0.0], 2: [0.5, 0.8, 1.5, 0.0, 0.0],
         3: [2.0, 0.5, 1.0, 1.0, 0.2], 4: [3.0, 1.0, 1.0, 3.0, 0.5]}[min(phase, 4)]
    losses['total'] = w[0]*losses['torsion'] + w[1]*losses['det'] + w[2]*losses['pos'] + w[3]*losses['harm'] + w[4]*losses['rg']
    losses['T_val'] = T_mean.item()
    losses['det_val'] = det_g.mean().item()
    return losses

print('Losses ready')

Losses ready


## 11. Checkpointing

In [11]:
def save_ckpt(net, rg, opt, phase, epoch, hist, cfg, name='ckpt.pt'):
    d = Path(cfg['output_dir']) / 'checkpoints'
    d.mkdir(exist_ok=True)
    torch.save({'phase': phase, 'epoch': epoch, 'net': net.state_dict(),
                'rg': rg.state_dict(), 'opt': opt.state_dict(), 'hist': hist}, d / name)

def load_ckpt(net, rg, opt, cfg):
    p = Path(cfg['output_dir']) / 'checkpoints' / 'ckpt.pt'
    if p.exists():
        c = torch.load(p, map_location=device)
        net.load_state_dict(c['net'])
        rg.load_state_dict(c['rg'])
        opt.load_state_dict(c['opt'])
        return c['phase'], c['epoch'], c['hist']
    return 1, 0, []

def get_lr(epoch, cfg):
    w = cfg['warmup_epochs']
    if epoch < w:
        return cfg['lr_initial'] * (epoch + 1) / w
    return max(cfg['lr_min'], cfg['lr_initial'] * 0.95 ** ((epoch - w) // 500))

print('Checkpointing ready')

Checkpointing ready


## 12. Training Loop

In [12]:
def train(net, rg, geom, cfg, zpp):
    opt = optim.Adam(list(net.parameters()) + list(rg.parameters()), lr=cfg['lr_initial'])
    phase0, epoch0, hist = load_ckpt(net, rg, opt, cfg)

    print('Training v1.3...')
    print('Phase | Epoch | T | det | Loss')

    for phase in range(phase0, 5):
        e0 = epoch0 if phase == phase0 else 0
        for epoch in range(e0, cfg['epochs_per_phase']):
            lr = get_lr(epoch, cfg)
            for pg in opt.param_groups:
                pg['lr'] = lr

            coords = sample_coords(cfg['batch_size'])
            losses = compute_losses(net, geom, coords, rg, zpp, phase)

            opt.zero_grad()
            losses['total'].backward()
            opt.step()

            if epoch % cfg['print_every'] == 0:
                print(f"  {phase} | {epoch:5d} | {losses['T_val']:.4f} | {losses['det_val']:.4f} | {losses['total'].item():.2e}")

            if epoch % cfg['checkpoint_every'] == 0 and epoch > 0:
                save_ckpt(net, rg, opt, phase, epoch, hist, cfg)

            hist.append({'phase': phase, 'epoch': epoch, 'loss': losses['total'].item(),
                         'T': losses['T_val'], 'det': losses['det_val']})

        save_ckpt(net, rg, opt, phase + 1, 0, hist, cfg)

    return pd.DataFrame(hist)

print('Training loop ready')

Training loop ready


## 13. TauScaleBridge

In [13]:
class TauScaleBridge:
    """TAU-SCALE: Map geometric integrals to masses using only structural constants."""

    def __init__(self, sc: StructuralConstants, zpp: ZeroParamPhysics):
        self.sc = sc
        self.tau = zpp.tau

    def mass_ratios(self) -> Dict:
        sc = self.sc
        return {
            'm_tau/m_e': sc.dim_K7 + 10 * sc.dim_E8 + 10 * sc.H_star,  # 3477
            'm_mu/m_e': sc.b3_target + sc.H_star + sc.M5,  # 207
            'm_s/m_d': 20,
            'tau': self.tau,
        }

    def print_predictions(self):
        r = self.mass_ratios()
        print('Zero-Parameter Mass Predictions:')
        print(f"  m_tau/m_e = {r['m_tau/m_e']} (exp: 3477.18)")
        print(f"  m_mu/m_e = {r['m_mu/m_e']} (exp: 206.77)")
        print(f"  m_s/m_d = {r['m_s/m_d']} (exp: ~20)")

print('TauScaleBridge ready')

TauScaleBridge ready


## 14. Initialize and Train

In [14]:
phi_net = PhiNet(TRAIN_CONFIG).to(device)
rg_module = RGFlowModule().to(device)
geometry = GeometryG2(TRAIN_CONFIG, SC)

print(f'Parameters: {sum(p.numel() for p in phi_net.parameters()):,}')

history_df = train(phi_net, rg_module, geometry, TRAIN_CONFIG, ZPP)
print('Training complete!')

Parameters: 374,051
Training v1.3...
Phase | Epoch | T | det | Loss
  1 |     0 | 0.0431 | 1.0545 | 4.47e-01
  1 |   100 | 0.0565 | 1.9321 | 3.79e-03
  1 |   200 | 0.0364 | 2.0003 | 2.00e-04
  1 |   300 | 0.0295 | 2.0001 | 8.63e-05
  1 |   400 | 0.0260 | 1.9990 | 4.64e-05
  1 |   500 | 0.0236 | 1.9981 | 2.75e-05
  1 |   600 | 0.0216 | 2.0009 | 1.42e-05
  1 |   700 | 0.0206 | 2.0005 | 8.77e-06
  1 |   800 | 0.0196 | 1.9996 | 5.19e-06
  1 |   900 | 0.0189 | 2.0005 | 3.30e-06
  1 |  1000 | 0.0183 | 2.0011 | 2.45e-06
  1 |  1100 | 0.0179 | 1.9993 | 1.31e-06
  1 |  1200 | 0.0174 | 1.9987 | 1.35e-06
  1 |  1300 | 0.0173 | 2.0025 | 3.42e-06
  1 |  1400 | 0.0171 | 2.0003 | 2.72e-07
  1 |  1500 | 0.0167 | 2.0015 | 1.16e-06
  1 |  1600 | 0.0166 | 1.9994 | 2.31e-07
  1 |  1700 | 0.0167 | 2.0016 | 1.26e-06
  1 |  1800 | 0.0166 | 2.0011 | 5.75e-07
  1 |  1900 | 0.0165 | 2.0003 | 5.32e-08
  2 |     0 | 0.0168 | 1.9991 | 6.94e-07
  2 |   100 | 0.0164 | 1.9999 | 8.92e-09
  2 |   200 | 0.0168 | 1.9996 

## 15. Analysis

In [15]:
# Harmonic check
print(f'b2 = {SC.b2_target} (target: 21)')
print(f'b3 = {SC.b3_target} (target: 77)')
print(f'H* = {SC.H_star}')

# Yukawa predictions
bridge = TauScaleBridge(SC, ZPP)
bridge.print_predictions()

b2 = 21 (target: 21)
b3 = 77 (target: 77)
H* = 99
Zero-Parameter Mass Predictions:
  m_tau/m_e = 3477 (exp: 3477.18)
  m_mu/m_e = 207 (exp: 206.77)
  m_s/m_d = 20 (exp: ~20)


## 16. Export

In [16]:
out = Path(TRAIN_CONFIG['output_dir']) / 'exports'
out.mkdir(parents=True, exist_ok=True)

# PyTorch
torch.save({'net': phi_net.state_dict(), 'rg': rg_module.state_dict()}, out / 'models.pt')

# NumPy
with torch.no_grad():
    coords = sample_coords(1000)
    g, info = geometry.compute_metric(phi_net, coords)
np.save(out / 'coords.npy', coords.cpu().numpy())
np.save(out / 'metric.npy', g.cpu().numpy())

# JSON
meta = {'kappa_T': ZPP.kappa_T, 'tau': ZPP.tau, 'b2': SC.b2_target, 'b3': SC.b3_target}
with open(out / 'metadata.json', 'w') as f:
    json.dump(meta, f, indent=2)

# CSV
history_df.to_csv(out / 'history.csv', index=False)

# LaTeX
tex = f'kappa_T = {ZPP.kappa_T:.6f}, tau = {ZPP.tau:.6f}'
with open(out / 'results.tex', 'w') as f:
    f.write(tex)

print(f'Exports saved to {out}')

Exports saved to outputs_v1_3/exports


## 17. Summary

In [17]:
print('=' * 60)
print('K7 GIFT v1.3 ZERO-PARAMETER SUMMARY')
print('=' * 60)
print(f'Structural: p2={SC.p2}, N_gen={SC.N_gen}, Weyl={SC.Weyl_factor}')
print(f'Topology: b2={SC.b2_target}, b3={SC.b3_target}, H*={SC.H_star}')
print(f'kappa_T = 1/61 = {ZPP.kappa_T:.6f}')
print(f'sin2_theta_W = 21/91 = {ZPP.sin2_theta_W:.6f}')
print(f'alpha_s = sqrt(2)/12 = {ZPP.alpha_s_MZ:.6f}')
print(f'tau = {ZPP.tau_num}/{ZPP.tau_den} = {ZPP.tau:.6f}')
print('=' * 60)

K7 GIFT v1.3 ZERO-PARAMETER SUMMARY
Structural: p2=2, N_gen=3, Weyl=5
Topology: b2=21, b3=77, H*=99
kappa_T = 1/61 = 0.016393
sin2_theta_W = 21/91 = 0.230769
alpha_s = sqrt(2)/12 = 0.117851
tau = 3472/891 = 3.896745
